In [1]:
import json
from IPython.display import clear_output, HTML, display
import ipywidgets.widgets as widgets

import sys
sys.path.append('/home/ballsbot/projects/ballsbot/python/lib')

import ballsbot.config as config
config.ENABLE_MULTIPROCESSING = False

In [2]:
import ballsbot.drawing as drawing
from ballsbot.lidar import calibration_to_xywh
from ballsbot.drawing import BotPoseAbsCoordsDrawing
from ballsbot.utils import keep_rps
from ballsbot.lidar import apply_transformation_to_cloud
from ballsbot.dashboard import Dashboard

In [3]:
only_nearby_meters = 4

In [4]:
new_format = True
with open('track_2024-08-20.json') as hf:
    if new_format:
        track = []
        for line in hf:
            try:
                frame = json.loads(line)
                track.append(frame)
            except Exception as e:
                print(f'broken frame {e}')
    else:
        track = json.loads(hf.read())

In [5]:
calibration = {
    'angle_fix': 3.0810609316737434,  # lidar orientation (radians)
    'fl_x': 0.2041686441507201,  # front left corner coords
    'fl_y': 0.10426277741236079,
    'rr_x': -0.08011094659163859,  # rear right corner coords
    'rr_y': -0.0988675829003773,
}
self_position = calibration_to_xywh(calibration)


dashboard = Dashboard(widgets, image_size=1200)

raw_painter = BotPoseAbsCoordsDrawing(dashboard, 'raw')
dashboard.add_subplot('raw', raw_painter.get_drawing_func_name())

aligned_painter = BotPoseAbsCoordsDrawing(dashboard, 'aligned')
dashboard.add_subplot('aligned', aligned_painter.get_drawing_func_name())

display(dashboard.get_image())

Image(value=b'', format='jpeg', height='1200', width='1200')

In [6]:
from python_pcl_ndt import python_pcl_ndt

In [7]:
raw_poses = []
raw_additional_points = []
aligned_poses = []
aligned_additional_points = []
ts = None
for frame_index, track_frame in enumerate(track):
    if frame_index == 11:
        break
    if not track_frame['tracker']['current_pose']:
        continue
    ts = keep_rps(ts, fps=0.25)
    print(frame_index)
    raw_poses.append(track_frame['tracker']['current_pose'])

    if track_frame['lidar']['augmented_points']:
        raw_points = apply_transformation_to_cloud(
            track_frame['lidar']['augmented_points'],
            [track_frame['tracker']['current_pose'][v] for v in ('x', 'y', 'teta')]
        )
        raw_additional_points += [x for i, x in enumerate(raw_points) if i % 10 == 0]
        raw_painter.update_image(
            raw_poses,
            raw_points,
            self_position,
            only_nearby_meters=only_nearby_meters,
            additional_points=raw_additional_points,
        )
        
        result = python_pcl_ndt.align_clouds(
            track_frame['tracker']['current_pose'],
            track_frame['lidar']['augmented_points'],
            True  # fast
        )
        aligned_poses.append({
            'x': result['x'],
            'y': result['y'],
            'teta': result['teta'],
        })
        aligned_points = apply_transformation_to_cloud(
            track_frame['lidar']['augmented_points'],
            [aligned_poses[-1][v] for v in ('x', 'y', 'teta')]
        )
        aligned_additional_points += [x for i, x in enumerate(aligned_points) if i % 10 == 0]
        aligned_painter.update_image(
            aligned_poses,
            aligned_points,
            self_position,
            only_nearby_meters=only_nearby_meters,
            additional_points=aligned_additional_points,
        )
        
        dashboard.redraw()
    else:
        aligned_poses.append(track_frame['tracker']['current_pose'])

1
2
3
4
5
6
7
8
9
10
